In [1]:
import openeo

In [2]:
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

Authenticated using refresh token.


In [3]:
# This will be required to import onnxruntime in the UDF.
dependencies_url = "https://artifactory.vgt.vito.be:443/auxdata-public/openeo/onnx_dependencies_1.16.3.zip"
# You can upload your own model to a public download link and paste the url here. An example of a public storage is github, or you can set up a public S3 bucket.
model_url = "https://artifactory.vgt.vito.be:443/auxdata-public/openeo/test_onnx_model.zip"

spatial_extent = {"west": 8.908, "south": 53.791, "east": 8.96, "north": 54.016}
temporal_extent = ["2022-10-01", "2022-12-01"]

In [4]:
s2_cube = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=temporal_extent,
    spatial_extent=spatial_extent,
    bands=["B04"],
    max_cloud_cover=20,
)

In [5]:
# Load the UDF from a file.
udf = openeo.UDF.from_file(
    "onnx_udf.py",
    context=None, # you can pass extra arguments to the UDF here
)

# Apply the UDF to the data cube.
onnx_result = s2_cube.apply_neighborhood(
    udf,
    size=[
        {"dimension": "x", "value": 256, "unit": "px"},
        {"dimension": "y", "value": 256, "unit": "px"},
    ],
    overlap=[
        {"dimension": "x", "value": 0, "unit": "px"},
        {"dimension": "y", "value": 0, "unit": "px"}
    ],
)

# We pass the model and dependencies as zip files to the UDF through the job options.
# The zip files provided in this list will be extracted in the UDF environment in a folder indicated by the name after the # symbol.
job_options = {
    "udf-dependency-archives": [
        f"{dependencies_url}#onnx_deps",
        f"{model_url}#onnx_models",
    ],
}
onnx_result.execute_batch("output.nc", job_options=job_options)

0:00:00 Job 'j-240612433033446b91231bcbf0cc8099': send 'start'
0:00:39 Job 'j-240612433033446b91231bcbf0cc8099': running (progress N/A)
0:00:45 Job 'j-240612433033446b91231bcbf0cc8099': running (progress N/A)
0:00:51 Job 'j-240612433033446b91231bcbf0cc8099': running (progress N/A)
0:01:00 Job 'j-240612433033446b91231bcbf0cc8099': running (progress N/A)
0:01:28 Job 'j-240612433033446b91231bcbf0cc8099': Connection error while polling job status: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
0:01:59 Job 'j-240612433033446b91231bcbf0cc8099': running (progress N/A)
0:02:12 Job 'j-240612433033446b91231bcbf0cc8099': running (progress N/A)
0:02:29 Job 'j-240612433033446b91231bcbf0cc8099': running (progress N/A)
0:02:49 Job 'j-240612433033446b91231bcbf0cc8099': running (progress N/A)
0:03:13 Job 'j-240612433033446b91231bcbf0cc8099': running (progress N/A)
0:03:45 Job 'j-240612433033446b91231bcbf0cc8099': error (progress N/A)
Your batch job 'j-24061

JobFailedException: Batch job 'j-240612433033446b91231bcbf0cc8099' didn't finish successfully. Status: error (after 0:03:48).